In [31]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Load and explore dataset
df = pd.read_csv('./dataset.csv', index_col=0)
df.index.name = 'Metric'


print(f"• Dataset Period: {df.columns[0]} to {df.columns[-1]} ({len(df.columns)} months)")
print(f"• Metrics Tracked: {len(df)} rows")
print(f"• Data Points: {len(df) * len(df.columns)}")


• Dataset Period: 2024 Jan to 2025 Sep (21 months)
• Metrics Tracked: 8 rows
• Data Points: 168


In [32]:
df.head()

,2024 Jan,2024 Feb,2024 Mar,2024 Apr,2024 May,2024 Jun,2024 Jul,2024 Aug,2024 Sep,2024 Oct,...,2024 Dec,2025 Jan,2025 Feb,2025 Mar,2025 Apr,2025 May,2025 Jun,2025 Jul,2025 Aug,2025 Sep
Metric,,,,,,,,,,,,,,,,,,,,,
Session - English pages,430914,413597,477633,416404,410077,385910,406635,397692,368259,388588,...,388468,450974,356171,355419,399918,324405,283366,347786,356629,306380
Session - Translated pages,161759,157467,235442,242864,259159,258653,287667,296229,307599,354334,...,375492,452859,386184,383492,384761,321777,263510,328569,351504,320145
AOC - English pages,120131,106011,124206,115356,115617,108125,111279,113361,101584,98504,...,96099,114716,81307,76381,77845,57568,52967,68324,73757,70171
AOC - Translated pages,25542,23909,31994,34506,39193,39457,44077,50636,61089,75329,...,78398,98215,79004,74758,73372,55519,46779,63612,72579,67693
CPA revenue - English pages,59076,57346,58629,63068,56893,60860,58242,63858,69882,73101,...,74513,91683,78684,57265,72455,56982,55739,49042,46597,57617


---
## DATA CLEANING & PREPARATION

In [33]:
print(f"{df.isnull().sum().sum()}")

0


The data set is a simple 10 Rows, so no need for Deep cleaning since no null/undefined values have been found

In [34]:
sessions_eng = df.loc['Session - English pages'].values
sessions_trans = df.loc['Session - Translated pages'].values
aoc_eng = df.loc['AOC - English pages'].values
aoc_trans = df.loc['AOC - Translated pages'].values
cpa_eng = df.loc['CPA revenue - English pages'].values
cpa_trans = df.loc['CPA revenue - Translated pages'].values
cpc_eng = df.loc['CPC revenue - English pages'].values
cpc_trans = df.loc['CPC revenue - Translated pages'].values

months = df.columns.tolist()
total_sessions = sessions_eng + sessions_trans
total_aoc = aoc_eng + aoc_trans
total_revenue = cpa_eng + cpa_trans + cpc_eng + cpc_trans

# Calculate efficiency metrics
ctr = (total_aoc / total_sessions) * 100  # Click-through rate
rev_per_session = total_revenue / total_sessions  # Revenue per session
rev_per_click = total_revenue / total_aoc  # Revenue per click
conv_rate = (total_aoc / total_sessions) * 100  # Conversion rate


In [35]:

metrics_summary = {
    'Total Website Sessions': f"{int(total_sessions.sum()):,}",
    'Total Account Open Clicks': f"{int(total_aoc.sum()):,}",
    'Total Revenue': f"${int((cpa_eng.sum() + cpa_trans.sum() + cpc_eng.sum() + cpc_trans.sum())):,}",
    'Avg Sessions per Month': f"{int(total_sessions.mean()):,}",
    'Avg AOC per Month': f"{int(total_aoc.mean()):,}",
    'Avg Revenue per Month': f"${int((cpa_eng + cpa_trans + cpc_eng + cpc_trans).mean()):,}",
    'Overall CTR': f"{ctr.mean():.2f}%",
    'Avg Revenue per Session': f"${rev_per_session.mean():.2f}",
    'Avg Revenue per Click': f"${rev_per_click.mean():.2f}",
    'Peak Monthly Revenue': f"${int((cpa_eng + cpa_trans + cpc_eng + cpc_trans).max()):,}"
}
summary_df = pd.DataFrame(list(metrics_summary.items()), columns=['Metric', 'Value'])
print(summary_df.to_string(index=False))


                   Metric      Value
   Total Website Sessions 14,618,702
Total Account Open Clicks  3,206,490
            Total Revenue $6,455,709
   Avg Sessions per Month    696,128
        Avg AOC per Month    152,690
    Avg Revenue per Month   $307,414
              Overall CTR     21.90%
  Avg Revenue per Session      $0.44
    Avg Revenue per Click      $2.05
     Peak Monthly Revenue   $376,230


---
## REVENUE  ANALYSIS (CPA vs CPC)

In [36]:
print("\nRevenue breakdown:")

total_cpa = cpa_eng.sum() + cpa_trans.sum()
total_cpc = cpc_eng.sum() + cpc_trans.sum()
total_rev = total_cpa + total_cpc

print(f"\nTotal:")
print(f"   CPC: ${int(total_cpc):,} ({(total_cpc/total_rev*100):.1f}%)")
print(f"   CPA: ${int(total_cpa):,} ({(total_cpa/total_rev*100):.1f}%)")




Revenue breakdown:

Total:
   CPC: $4,499,507 (69.7%)
   CPA: $1,956,202 (30.3%)


In [37]:

# Pie chart for revenue breakdown
fig_revenue_breakdown = go.Figure(data=[
    go.Pie(
        labels=['CPC', 'CPA'],
        values=[total_cpc, total_cpa],
        marker=dict(colors=['#1f77b4', '#2ca02c']),
        textinfo='label+percent+value',
        hovertemplate='<b>%{label}</b><br>%{value:,}<br>%{percent}<extra></extra>'
    )
])

fig_revenue_breakdown.update_layout(
    title='Total Revenue Breakdown: CPC vs CPA',
    height=500,
    template='plotly_white'
)
fig_revenue_breakdown.show()

In [38]:

# Line chart for CPA and CPC over time
total_cpa_monthly = cpa_eng + cpa_trans
total_cpc_monthly = cpc_eng + cpc_trans

fig_cpa_cpc_trend = go.Figure()

fig_cpa_cpc_trend.add_trace(go.Scatter(
    x=months,
    y=total_cpc_monthly,
    mode='lines+markers',
    name='CPC',
    line=dict(color='#1f77b4', width=2)
))

fig_cpa_cpc_trend.add_trace(go.Scatter(
    x=months,
    y=total_cpa_monthly,
    mode='lines+markers',
    name='CPA',
    line=dict(color='#2ca02c', width=2)
))

fig_cpa_cpc_trend.update_layout(
    title='Revenue Trend: CPA vs CPC Over Time',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)
fig_cpa_cpc_trend.show()

In [39]:
# CPA vs CPC Stacked
fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=months,
    y=cpc_eng,
    name='English CPC',
    marker=dict(color='#1f77b4')
))

fig3.add_trace(go.Bar(
    x=months,
    y=cpa_eng,
    name='English CPA',
    marker=dict(color='#2ca02c')
))

fig3.add_trace(go.Bar(
    x=months,
    y=cpc_trans,
    name='Translated CPC',
    marker=dict(color='#ff7f0e')
))

fig3.add_trace(go.Bar(
    x=months,
    y=cpa_trans,
    name='Translated CPA',
    marker=dict(color='#d62728')
))

fig3.update_layout(
    barmode='stack',
    title='Revenue Analysis: CPA vs CPC Distribution',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)
fig3.show()

In [40]:
# Calculate growth metrics between first and last month
first_month_sessions = total_sessions[0]
last_month_sessions = total_sessions[-1]
first_month_aoc = total_aoc[0]
last_month_aoc = total_aoc[-1]
first_month_revenue = total_revenue[0]
last_month_revenue = total_revenue[-1]

# Calculate percentage growth
sessions_growth = ((last_month_sessions - first_month_sessions) / first_month_sessions) * 100
aoc_growth = ((last_month_aoc - first_month_aoc) / first_month_aoc) * 100
revenue_growth = ((last_month_revenue - first_month_revenue) / first_month_revenue) * 100

metrics_growth = {
    'Sessions Growth': sessions_growth,
    'AOC Growth': aoc_growth,
    'Revenue Growth': revenue_growth
}


---
## REVENUE ANALYSIS (English vs Transulated)

In [42]:

fig1 = go.Figure()

eng_total_rev_monthly = cpa_eng + cpc_eng
trans_total_rev_monthly = cpa_trans + cpc_trans

fig1.add_trace(go.Scatter(
    x=months,
    y=eng_total_rev_monthly,
    mode='lines+markers',
    name='English Pages',
    line=dict(color='#1f77b4', width=3),
    fill='tozeroy',
    fillcolor='rgba(31, 119, 180, 0.1)'
))

fig1.add_trace(go.Scatter(
    x=months,
    y=trans_total_rev_monthly,
    mode='lines+markers',
    name='Translated Pages',
    line=dict(color='#ff7f0e', width=3),
    fill='tozeroy',
    fillcolor='rgba(255, 127, 14, 0.1)'
))

fig1.update_layout(
    title='Revenue Comparison CPC+CPA for English vs Translated Pages',
    xaxis_title='Month',
    yaxis_title='Revenue ($)',
    hovermode='x unified',
    height=500,
    template='plotly_white',
    font=dict(size=12),
    showlegend=True
)
fig1.show()

---
## Mini Dashboard


In [43]:
chart = go.Figure()

chart.add_trace(go.Scatter(
    x=months, y=sessions_eng,
    mode='lines+markers', name='English',
    line=dict(color='#1f77b4', width=2)
))

chart.add_trace(go.Scatter(
    x=months, y=sessions_trans,
    mode='lines+markers', name='Translated',
    line=dict(color='#ff7f0e', width=2)
))

chart.update_layout(
    title=' Sessions Revenue comparison: English vs translated pages',
    xaxis_title='Month', yaxis_title='Sessions',
    height=400, template='plotly_white',
    hovermode='x unified'
)
chart.show()


In [44]:
eng_revenue = cpa_eng + cpc_eng
trans_revenue = cpa_trans + cpc_trans

chart2 = go.Figure()

chart2.add_trace(go.Bar(
    x=months, y=eng_revenue,
    name='English', marker=dict(color='#1f77b4')
))

chart2.add_trace(go.Bar(
    x=months, y=trans_revenue,
    name='Translated', marker=dict(color='#ff7f0e')
))

chart2.update_layout(
    barmode='group',
    title='Revenue (CPA + CPC) - English vs Translated',
    xaxis_title='Month', yaxis_title='Revenue ($)',
    height=400, template='plotly_white',
    hovermode='x unified'
)
chart2.show()

print(f"\n2024-2025 Revenue Totals:")
print(f"   English: ${int(eng_revenue.sum()):,}")
print(f"   Translated: ${int(trans_revenue.sum()):,}")


2024-2025 Revenue Totals:
   English: $4,149,301
   Translated: $2,306,408


In [45]:
eng_rev_per_session = eng_revenue / sessions_eng
trans_rev_per_session = trans_revenue / sessions_trans

chart3 = go.Figure()

chart3.add_trace(go.Scatter(
    x=months, y=eng_rev_per_session,
    mode='lines+markers', name='English',
    line=dict(color='#1f77b4', width=2)
))

chart3.add_trace(go.Scatter(
    x=months, y=trans_rev_per_session,
    mode='lines+markers', name='Translated',
    line=dict(color='#ff7f0e', width=2)
))

chart3.update_layout(
    title='Revenue per Session',
    xaxis_title='Month', yaxis_title='Revenue per Session ($)',
    height=400, template='plotly_white',
    hovermode='x unified'
)
chart3.show()